# Optimize Lambda with Locust and SageMaker HPO

## Setup environment

In [ ]:
!pip install -q sagemaker==2.33.0

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch as PyTorchEstimator
from sagemaker.tuner import IntegerParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
base_name = 'lambda-tuning'
container = '<your-lambda-container-image-uri'
model_s3_uri = '<your-model-binary-s3-path>'

## Set static hyperparameters, metrics, and estimator

In [ ]:
# Dictionary of all hyperparameters and variable we want to pass to our training image:
hyperparameters={
    'container': container,
    'model_s3_uri': model_s3_uri,
    'role': role,
    'region': region
}

# We want to capture the following metric for tuning hyperparameters
metric_definitions = [
    {
        'Name': 'aggregate_score',
        'Regex': 'AGGREGATE SCORE: ([0-9]+)'
    }
]

# We define the Estimator object with the provided pytorch container)
estimator = PyTorchEstimator(
    entry_point='entry_point.py',
    source_dir='source_dir',
    hyperparameters=hyperparameters,
    metric_definitions=metric_definitions,
    role=role,
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    framework_version='1.8.0',
    py_version='py3',
    base_job_name=base_name,
    disable_profiler=True
)

## Set hyperparameter ranges

In [ ]:
hyperparameter_ranges = {
    'lambda_memory': IntegerParameter(128, 9728)
}

objective_metric_name = 'aggregate_score'
objective_type = 'Minimize'

## Launch tuning job

In [ ]:
tuner = HyperparameterTuner(
    estimator=estimator,
    base_tuning_job_name=base_name,
    metric_definitions=metric_definitions,
    objective_metric_name=objective_metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=40,
    max_parallel_jobs=5
)

tuner.fit()